# Daily Snapshots Time Series Plotter

This notebook loads `{TEAM_ID}/daily_snapshots.csv` from the QuantConnect Object Store and generates time series plots for all numeric columns.

**Prerequisites:** Run the WolfpackTrend backtest first to generate data in ObjectStore

**Features:**
- Auto-plot all numeric columns as time series
- Date filter via `start_date` / `end_date` variables
- Summary statistics for each column

In [ ]:
# Import QuantConnect research libraries
from QuantConnect import *
from QuantConnect.Research import QuantBook

qb = QuantBook()

# Import data and plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from io import StringIO
from config import TEAM_ID

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 5)

print("Libraries loaded successfully")
print(f"QuantBook initialized")

## Configuration

Set date filters here to focus on a specific time period.

In [ ]:
# Date filter configuration
# Set to None to use all available data
start_date = None  # e.g., "2022-06-01"
end_date = None    # e.g., "2023-12-31"

print(f"Date filter: {start_date or 'start'} to {end_date or 'end'}")

## Load Data from ObjectStore

In [ ]:
# Load CSV data from ObjectStore
print("Loading data from ObjectStore...")
print("=" * 60)

try:
    snapshots_str = qb.ObjectStore.Read(f"{TEAM_ID}/daily_snapshots.csv")
    df = pd.read_csv(StringIO(snapshots_str))
    
    # Parse date column if it exists
    if 'date' in df.columns:
        df['date'] = pd.to_datetime(df['date'])
        df = df.sort_values('date')
    
    print(f"Daily Snapshots: {len(df)} rows")
    print(f"Date range: {df['date'].min().strftime('%Y-%m-%d')} to {df['date'].max().strftime('%Y-%m-%d')}")
    print(f"Columns: {list(df.columns)}")
    print()
    print(df.head())
    print()
    print("=" * 60)
    print("Data loaded successfully!")
    
except Exception as e:
    print(f"ERROR: {str(e)}")
    print("\nMake sure you have run the WolfpackTrend backtest first to generate ObjectStore data.")
    df = pd.DataFrame()

## Apply Date Filter

In [ ]:
if not df.empty and 'date' in df.columns:
    df_filtered = df.copy()
    
    if start_date:
        df_filtered = df_filtered[df_filtered['date'] >= pd.to_datetime(start_date)]
    
    if end_date:
        df_filtered = df_filtered[df_filtered['date'] <= pd.to_datetime(end_date)]
    
    print(f"Filtered data: {len(df_filtered)} rows")
    if len(df_filtered) > 0:
        print(f"Date range: {df_filtered['date'].min().strftime('%Y-%m-%d')} to {df_filtered['date'].max().strftime('%Y-%m-%d')}")
    
    df = df_filtered
else:
    print("No data to filter")

## Identify Date and Numeric Columns

In [ ]:
if not df.empty:
    # Identify the date column
    date_column = None
    if 'date' in df.columns:
        date_column = 'date'
    else:
        # Find first datetime column
        for col in df.columns:
            if pd.api.types.is_datetime64_any_dtype(df[col]):
                date_column = col
                break
    
    if date_column is None:
        print("WARNING: No date column found. Using row index for x-axis.")
    else:
        print(f"Using '{date_column}' as x-axis for time series plots")
    
    # Identify numeric columns (excluding the date column)
    numeric_columns = []
    for col in df.columns:
        if col != date_column and pd.api.types.is_numeric_dtype(df[col]):
            numeric_columns.append(col)
    
    print(f"\nFound {len(numeric_columns)} numeric columns to plot:")
    for i, col in enumerate(numeric_columns, 1):
        print(f"  {i}. {col}")
else:
    print("No data available to plot")
    date_column = None
    numeric_columns = []

## Generate Time Series Plots

In [ ]:
if numeric_columns:
    print("Generating time series plots...")
    print("=" * 60)
    
    # Determine x-axis data
    if date_column:
        x_data = df[date_column]
        x_label = date_column.capitalize()
    else:
        x_data = df.index
        x_label = "Row Index"
    
    # Plot each numeric column
    for col in numeric_columns:
        plt.figure(figsize=(14, 5))
        
        # Plot the time series
        plt.plot(x_data, df[col], linewidth=2, color='steelblue')
        
        # Add formatting
        plt.title(f"Time Series: {col}", fontsize=14, fontweight='bold')
        plt.xlabel(x_label, fontsize=12)
        plt.ylabel(col, fontsize=12)
        plt.grid(True, alpha=0.3)
        
        # Rotate x-axis labels if dates
        if date_column:
            plt.xticks(rotation=45, ha='right')
        
        # Add statistics annotation
        stats_text = f"Mean: {df[col].mean():.4f}\nStd: {df[col].std():.4f}\nMin: {df[col].min():.4f}\nMax: {df[col].max():.4f}"
        plt.text(0.02, 0.98, stats_text, 
                transform=plt.gca().transAxes,
                fontsize=9,
                verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
        
        plt.tight_layout()
        plt.show()
        
        print(f"Plotted: {col}")
    
    print("=" * 60)
    print(f"All {len(numeric_columns)} plots generated successfully!")
else:
    print("No numeric columns available to plot")

## NAV and Cumulative P&L

In [ ]:
if not df.empty and 'nav' in df.columns:
    fig, axes = plt.subplots(2, 1, figsize=(14, 10), sharex=True)
    
    # NAV over time
    axes[0].plot(df['date'], df['nav'], linewidth=2, color='steelblue')
    axes[0].set_title('NAV Over Time', fontsize=14, fontweight='bold')
    axes[0].set_ylabel('NAV ($)', fontsize=12)
    axes[0].grid(True, alpha=0.3)
    
    # Cumulative P&L if available
    if 'cumulative_pnl' in df.columns:
        axes[1].plot(df['date'], df['cumulative_pnl'], linewidth=2, color='green')
        axes[1].fill_between(df['date'], 0, df['cumulative_pnl'], 
                             where=df['cumulative_pnl'] >= 0, alpha=0.3, color='green')
        axes[1].fill_between(df['date'], 0, df['cumulative_pnl'], 
                             where=df['cumulative_pnl'] < 0, alpha=0.3, color='red')
        axes[1].set_title('Cumulative P&L Over Time', fontsize=14, fontweight='bold')
        axes[1].set_ylabel('Cumulative P&L ($)', fontsize=12)
        axes[1].axhline(y=0, color='k', linestyle='-', alpha=0.3)
        axes[1].grid(True, alpha=0.3)
    else:
        # Compute cumulative P&L from daily returns
        df['returns'] = df['nav'].pct_change()
        df['cumulative_returns'] = (1 + df['returns']).cumprod() - 1
        axes[1].plot(df['date'], df['cumulative_returns'] * 100, linewidth=2, color='green')
        axes[1].set_title('Cumulative Returns (%)', fontsize=14, fontweight='bold')
        axes[1].set_ylabel('Cumulative Return (%)', fontsize=12)
        axes[1].axhline(y=0, color='k', linestyle='-', alpha=0.3)
        axes[1].grid(True, alpha=0.3)
    
    axes[1].set_xlabel('Date', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()
else:
    print("NAV column not found in data")

## Exposure Over Time

In [ ]:
# Check for exposure columns
exposure_cols = [c for c in df.columns if 'exposure' in c.lower()]

if exposure_cols and not df.empty:
    fig, ax = plt.subplots(figsize=(14, 6))
    
    for col in exposure_cols:
        ax.plot(df['date'], df[col], linewidth=2, label=col)
    
    ax.set_title('Portfolio Exposure Over Time', fontsize=14, fontweight='bold')
    ax.set_xlabel('Date', fontsize=12)
    ax.set_ylabel('Exposure', fontsize=12)
    ax.legend(loc='upper left')
    ax.axhline(y=0, color='k', linestyle='-', alpha=0.3)
    ax.grid(True, alpha=0.3)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()
else:
    print("No exposure columns found in data")

## Summary Statistics Table

In [ ]:
if numeric_columns:
    print("\n" + "=" * 60)
    print("SUMMARY STATISTICS FOR ALL NUMERIC COLUMNS")
    print("=" * 60)
    
    # Generate summary statistics
    summary = df[numeric_columns].describe().T
    
    # Add additional statistics
    summary['median'] = df[numeric_columns].median()
    summary['skew'] = df[numeric_columns].skew()
    summary['kurtosis'] = df[numeric_columns].kurtosis()
    
    # Reorder columns
    summary = summary[['count', 'mean', 'median', 'std', 'min', '25%', '50%', '75%', 'max', 'skew', 'kurtosis']]
    
    print(summary.to_string())
    print("\n" + "=" * 60)
else:
    print("No numeric columns available for statistics")